In [1]:
# libraries
import turtle
from turtle import *
from random import randrange
from freegames import square, vector

In [2]:
# functions
# score
def updateScoreTurtle():
    pen.clear()
    scoretext ="Score: %s" % score
    pen.write(scoretext, 
              align="Center", 
              font=("Comic Sans", 24, "normal"))

# help save obstacles to a list of dictionaries
def add_obs(ob_dict,obs_x,obs_y,unitHt,unitWt):
    u = 10
    ob_dict["x_val"] = obs_x
    ob_dict["y_val"] = obs_y
    ob_dict["ht_val"] = unitHt
    ob_dict["wt_val"] = unitWt
    ob_dict["x_range"] = obs_x+unitWt*u
    ob_dict["y_range"] = obs_y+unitHt*u
    obs_list.append(ob_dict)

# hit obstacle 
def hit_obstacle(head):
    "Return True if head hits an obstacle."
    for ob in obs_list:
        ob_x_range = range(ob["x_val"],ob["x_range"])
        ob_y_range = range(ob["y_val"],ob["y_range"])
        
        if head.x in ob_x_range and head.y in ob_y_range:
            return True
    return False
#     ob1_x_range = range(-100, 1)  # x-coordinate range of ob1 obstacle
#     ob1_y_range = range(-10, 11)  # y-coordinate range of ob1 obstacle

#     return head.x in ob1_x_range and head.y in ob1_y_range

# base: updates aim vector with new direction 
def change(x, y):
    "Change snake direction."
    aim.x = x
    aim.y = y

# base: boundary of game
def inside(head):
    "Return True if head inside boundaries."
    return -200 < head.x < 190 and -200 < head.y < 190

def move():
    
    global blue_food
    global blue_food_active
    
    global orange_food
    global orange_food_active 
    
    global speed_factor
    global score
    
    
    # base: draw head, move in aim direction
    "Move snake forward one segment."
    head = snake[-1].copy()
    head.move(aim)

    # base: not inside boundaries or head hit myself
    if not inside(head) or head in snake or hit_obstacle(head):
        ##CHANGED
        square(head.x, head.y, 10, 'red')
        update()
        game_end()
        return #stop move func n exit game loop !!!!!!!!!!

    # add new segment to snake body 
    snake.append(head)
    
    if head == food and not blue_food_active and not orange_food_active:
        print('Snake1:', len(snake)) #update to display current length of snake
        score += 50 
        print(score)
        updateScoreTurtle()
        food.x, food.y = coord_checker(obs_list)
    
        # probability 1/3 to random generate food
        food_type = randrange(0,3)
        if food_type == 2:  
            blue_food_x, blue_food_y = coord_checker(obs_list)
            blue_food = vector(blue_food_x, blue_food_y)
            blue_food_active = True
            
        elif food_type == 1:
            orange_food_x, orange_food_y = coord_checker(obs_list)
            orange_food = vector(orange_food_x, orange_food_y)
            orange_food_active = True
            
    elif head == blue_food:
        print('Snake2:', len(snake))
        score += 100 
        print(score)
        updateScoreTurtle()
        speed_factor = 3
        blue_food_active  = False
        ##CHANGED
        food.x, food.y = coord_checker(obs_list)
    
    elif head == orange_food: 
        print('Snake3:', len(snake))
        score += 200 
        print(score)
        updateScoreTurtle()
        speed_factor = 5
        orange_food_active  = False
        ##CHANGED
        food.x, food.y = coord_checker(obs_list)
        
        
    else:
        snake.pop(0) # remove the pixel at the end, illusion of movement 
        
    clear() #remove prev drawn body n update w new one 
    
    # draw arena
    draw_arena()
    
    # obstacle drawing function (implement for loop)
    #draw_obs(-100,0,2,5)
    for obs in obs_list:
        draw_obs(obs["x_val"],obs["y_val"],obs["ht_val"],obs["wt_val"])
    
    for index, body in enumerate(snake): #keep track of no of iterations aka snake segments
        if index == len(snake) - 1:
            ##CHANGED
            square(body.x, body.y, 10, 'black')
        else:
            ##CHANGED
            square(body.x, body.y, 10, 'grey')
        
    if not blue_food_active and not orange_food_active: 
        #square(food.x, food.y, 9, 'green')
        penup()
        goto(food.x+5, food.y+5)
        dot(10, 'green')
        
    elif blue_food_active: 
        penup()
        goto(blue_food.x+5, blue_food.y+5)
        dot(10, 'blue')
    
    elif orange_food_active: 
        penup()
        goto(orange_food.x+5 ,orange_food.y+5)
        dot(10, 'orange')

    update()
    ontimer(move, int(300 / speed_factor)) #delay 

# all the obstacles 
def draw_obs(x,y,unitHt,unitWt):
    u = 10
    for h in range(unitHt):
        for w in range(unitWt):
            square(x+u*w, y+u*h, u, 'pink')
            
def draw_arena():
    square(-200, -200, 400, 'purple')
    square(-190, -190, 380, 'white')

def coord_checker(obs_list):
    while True:
        x_food = randrange(-15, 15) * 10
        y_food = randrange(-15, 15) * 10
        inside_obs = False
        
        for obs in obs_list:
            ob_x_range = range(obs["x_val"], obs["x_range"])
            ob_y_range = range(obs["y_val"], obs["y_range"])

            if x_food+5 in ob_x_range and y_food+5 in ob_y_range:
                inside_obs = True
                break
            elif x_food-5 in ob_x_range and y_food-5 in ob_y_range:
                inside_obs = True
                break
                
        if not inside_obs:
            return x_food, y_food

In [3]:
# basic game setup
def game_setup():
    game_screen = turtle.Screen()
    game_screen.title("Snake! by Janine, Kaia, and Kelly")
    game_screen.bgcolor("white")
    game_screen.setup(800,600,370,0)
    hideturtle()
    tracer(False)

    global obs_list
    obs_list = []

    global blue_food
    global blue_food_active
    
    global orange_food
    global orange_food_active 
    global food
    global snake
    global aim
    food = vector(0, 0) # start for food 
    snake = [vector(10, 0)] # start for snake  
    aim = vector(0, -10) # where snake starts moving 

    # make them inactive to allow for the random generation to start aft green
    blue_food = None
    blue_food_active = False 
    orange_food = None 
    orange_food_active = False 

    # obstacles
    ob1 = {}
    add_obs(ob1,-150,0,3,6)
    ob2 = {}
    add_obs(ob2,100,50,6,3)
    ob3 = {}
    add_obs(ob3,50,-120,3,3)
    ob4 = {}
    add_obs(ob4,-100,100,6,3)
    ob5 = {}
    add_obs(ob5,-120,-150,3,6)

    global score
    score = 0
    global speed_factor
    speed_factor = 1.5 

    #scoreboard
    global pen
    pen = turtle.Turtle()
    pen.speed(0)
    pen.shape("square")
    pen.color("black")
    pen.penup()
    pen.hideturtle()
    pen.goto(0, 250)
    updateScoreTurtle()

def game_splash():
    global start_message
    start_message = turtle.Turtle()
    start_message.penup()
    start_message.hideturtle()
    start_message.goto(0, -200)
    start_message.write("""
    WELCOME TO THE SNAKE GAME ! 
    Eat the yummy food to grow,
    but beware of the obstacles, hit them and it's game over :<
    Point & Speed system:
    Green food - 100 points, normal speed 
    Blue food - 200 points, x3 speed buff
    Orange food - 500 points, x5 speed buff
    Movement keys:
    W / Up arrow - Up
    A / Left arrow - Left
    S / Down arrow - Down
    D / Right arrow - Right
    Press 'q' to start the game!
    """, 
                        align="center", 
                        font=("Arial", 20, "normal"))

def game_start():
    start_message.clear()
    game_setup()
    move()

def game_end():
    turtle.clear()
    pen.clear()
    start_message = turtle.Turtle()
    start_message.penup()
    start_message.hideturtle()
    start_message.goto(0, 0)
    start_message.write("Press 'q' to restart the game!", 
                        align="center", 
                        font=("Arial", 20, "normal"))

In [4]:
# base: start game
global game_state
game_splash()
listen() #listen to keyboard events 
onkey(game_start, 'q')
onkey(lambda: change(10, 0), 'd')
onkey(lambda: change(-10, 0), 'a')
onkey(lambda: change(0, 10), 'w')
onkey(lambda: change(0, -10), 's')
onkey(lambda: change(10, 0), 'Right')
onkey(lambda: change(-10, 0), 'Left')
onkey(lambda: change(0, 10), 'Up')
onkey(lambda: change(0, -10), 'Down')
# main game loop
done()

Snake1: 2
50
Snake2: 3
150
Snake1: 4
200
Snake1: 2
50
Snake1: 3
100
Snake2: 4
200
Snake1: 5
250
Snake1: 2
50
Snake2: 3
150
Snake1: 2
50
Snake3: 3
250
Snake1: 4
300
Snake1: 2
50
Snake1: 3
100
Snake2: 4
200
Snake1: 5
250
